<a href="https://colab.research.google.com/github/ense-nastydev/ChatAnalytics/blob/main/TGroup_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telegram chat analytics with Pandas

This notebook is still a work in progress. we can now proces the machine readable json from a chat export file created with Telegram desktop. 

The easiest way to get going is to open this notebook in colaboratory and upload your chat export file (result.json) to the created instance


# Setup

Install missing software and up to date libraries

In [ ]:
%%bash
pip install boltons more-itertools  


Also upload the data to work on to your instance

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print(
        'User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])
        )
    )

# Data Processing

make  lazy json loader and use it to feed a pandas dataframe.
This dataframe will get transformed in a variety o ways for analysis

### Helper Functions

In [ ]:
from datetime import datetime
from boltons.iterutils import windowed_iter


def gendeltas(timestamps_object, objname):
    """Generates a column with time deltas calculated from the current
    and previous timestamp value. First timedelta is always 0"""
    a = datetime.now()
    deltas = [a - a]
    for older, newer in windowed_iter(timestamps_object, 2):
        deltatime = datetime.fromisoformat(newer) - datetime.fromisoformat(older)
        if deltatime >= (a - a):
            deltas.append(deltatime)
        else:
            deltas.append(a - a)
    deltasS = pd.Series(
        deltas,
        name=objname,
    )
    return deltasS

In [ ]:
def cleandataframe(dataframeobj):
    """removes a bunch of atributes unneeded for analysis."""
    keepers = [
        "wait_time",
        "from",
        "date",
        "text",
        "photo",
        "id",
        "reply_to_message_id",
    ]

    def dropper(key):
        dataframeobj.drop(key, axis="columns", inplace=True)
        return key

    x = [dropper(key) for key in dataframeobj.columns if key not in keepers]
    return dataframeobj

#### Text analysis helper funcions

In [ ]:
def fixtype(msgdict):
    """augments the message data object with calculated values from
    the messages own data. This is necessary to have a correctly typed index
    so we cast datetime.datetime from elegrams mercifully chosen ISO date string
    We also fix the url in file and photo fields in order to be able to display
    this data, you may need to customize the output to your chosen storage
    facility, Currently set up for Google Cloud Storage buckets.
    """

    newmsg = msgdict.copy()
    newmsg["date"] = datetime.fromisoformat(msgdict["date"])
    if "photo" in msgdict.keys():
        newmsg["photo"] = "fixurl({})".format(msgdict["photo"])
    if "file" in msgdict.keys():
        newmsg["file"] = "fixurl({})".format(msgdict["file"])
    if "text" in msgdict.keys and len(msgdict > 0):
        newmsg["wordcount"] = text_stats(msgdict["text"])
    return newmsg


def fixurl(txt):
    ...
    return txt


def text_stats(txt):
    wordcount = len(txt.split(" "))
    ...
    return wordcount

### Main processing

In [ ]:
import json
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable


def data_generator():
    all_messages = []
    for f in uploaded:
        with open(f, "r", encoding="utf8") as chat:
            conversation = json.load(chat)
            [all_messages.append(msg) for msg in conversation["messages"]]
    return (msg for msg in all_messages)


def getcolumnvals(column):
    all_values = []
    for f in uploaded:
        with open(f, "r", encoding="utf8") as chat:
            conversation = json.load(chat)
            [all_values.append(msg) for msg in conversation["messages"]]
    return (val[column] for val in all_values)


messages = data_generator()
df = pd.DataFrame.from_records(
    [m for m in data_generator()], index="date", coerce_float=False
)
cleandataframe(df)

# deltas = gendeltas(df['date'],"wait_time")
df.info()

In [ ]:
idx = pd.Index([d for d in getcolumnvals("date")], name="tstamp")
df.info()

In [ ]:
df.join(deltas)
idx = pd.to_datetime([d for d in getcolumnvals("date")])
df.index
midx = pd.MultiIndex.from_arrays([idx, df.index], names=["tstamp", "n"])

# df.resample("D",on="date")

In [ ]:
import altair as alt

alt.Chart(df).mark_line().encode(x="date:T", y="count", color="symbol").interactive(
    bind_y=False
)

In [ ]:
from vega_datasets import data

stocks = data.stocks()

import altair as alt

alt.Chart(stocks).mark_line().encode(x="date:T", y="price", color="symbol").interactive(
    bind_y=False
)

In [ ]:
cdf.loc[:, ["date", "from", "text", "photo", "id", "reply_to_message_id"]]
tdf = idf.to_timestamp(copy=True)
tdf.info()